In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import cvxpy
import math
import numpy as np
import cubic_spline_planner


NX = 4  # x = x, y, v, yaw
NU = 2  # a = [accel, steer]
T = 5  # horizon length

# mpc parameters
R = np.diag([0.01, 0.01])  # input cost matrix
Rd = np.diag([0.01, 1.0])  # input difference cost matrix
Q = np.diag([1.0, 1.0, 0.5, 0.5])  # state cost matrix
Qf = Q  # state final matrix
GOAL_DIS = 1.5  # goal distance
STOP_SPEED = 0.5 / 3.6  # stop speed
MAX_TIME = 500.0  # max simulation time

# iterative paramter
MAX_ITER = 3  # Max iteration
DU_TH = 0.1  # iteration finish param

TARGET_SPEED = 10.0 / 3.6  # [m/s] target speed
N_IND_SEARCH = 10  # Search index number

DT = 0.2  # [s] time tick

# Vehicle parameters
LENGTH = 4.5  # [m]
WIDTH = 2.0  # [m]
BACKTOWHEEL = 1.0  # [m]
WHEEL_LEN = 0.3  # [m]
WHEEL_WIDTH = 0.2  # [m]
TREAD = 0.7  # [m]
WB = 110#2.5  # [m]

MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 1.0  # maximum accel [m/ss]

show_animation = True



        

def pi_2_pi(angle):
    while(angle > math.pi):
        angle = angle - 2.0 * math.pi

    while(angle < -math.pi):
        angle = angle + 2.0 * math.pi

    return angle





def plot_car(x, y, yaw, steer=0.0, cabcolor="-r", truckcolor="-k"):  # pragma: no cover

    outline = np.array([[-BACKTOWHEEL, (LENGTH - BACKTOWHEEL), (LENGTH - BACKTOWHEEL), -BACKTOWHEEL, -BACKTOWHEEL],
                        [WIDTH / 2, WIDTH / 2, - WIDTH / 2, -WIDTH / 2, WIDTH / 2]])

    fr_wheel = np.array([[WHEEL_LEN, -WHEEL_LEN, -WHEEL_LEN, WHEEL_LEN, WHEEL_LEN],
                         [-WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD]])

    rr_wheel = np.copy(fr_wheel)

    fl_wheel = np.copy(fr_wheel)
    fl_wheel[1, :] *= -1
    rl_wheel = np.copy(rr_wheel)
    rl_wheel[1, :] *= -1

    Rot1 = np.array([[math.cos(yaw), math.sin(yaw)],
                     [-math.sin(yaw), math.cos(yaw)]])
    Rot2 = np.array([[math.cos(steer), math.sin(steer)],
                     [-math.sin(steer), math.cos(steer)]])

    fr_wheel = (fr_wheel.T.dot(Rot2)).T
    fl_wheel = (fl_wheel.T.dot(Rot2)).T
    fr_wheel[0, :] += WB
    fl_wheel[0, :] += WB

    fr_wheel = (fr_wheel.T.dot(Rot1)).T
    fl_wheel = (fl_wheel.T.dot(Rot1)).T

    outline = (outline.T.dot(Rot1)).T
    rr_wheel = (rr_wheel.T.dot(Rot1)).T
    rl_wheel = (rl_wheel.T.dot(Rot1)).T

    outline[0, :] += x
    outline[1, :] += y
    fr_wheel[0, :] += x
    fr_wheel[1, :] += y
    rr_wheel[0, :] += x
    rr_wheel[1, :] += y
    fl_wheel[0, :] += x
    fl_wheel[1, :] += y
    rl_wheel[0, :] += x
    rl_wheel[1, :] += y

    plt.plot(np.array(outline[0, :]).flatten(),
             np.array(outline[1, :]).flatten(), truckcolor)
    plt.plot(np.array(fr_wheel[0, :]).flatten(),
             np.array(fr_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(rr_wheel[0, :]).flatten(),
             np.array(rr_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(fl_wheel[0, :]).flatten(),
             np.array(fl_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(rl_wheel[0, :]).flatten(),
             np.array(rl_wheel[1, :]).flatten(), truckcolor)
    plt.plot(x, y, "*")




def get_nparray_from_matrix(x):
    return np.array(x).flatten()


def get_closest_point(state, ccx, ccy, cyaw, pind):
    print(ccx,ccy)
    d = [(state.x-idx) ** 2 + (state.y-idy) ** 2 for (idx, idy) in zip(ccx,ccy)]
    print('printing d',d,len(d),type(d))
    mind = min(d)
    ind = d.index(mind) + pind
    print('ds',ind,mind)
    return ind, mind




def ccalc_ref_trajectory(state, cx, cy, cyaw, ck, sp, dl, pind):
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    ind, _ = get_closest_point(state, cx, cy, cyaw, pind)

    if pind >= ind:
        ind = pind

    xref[0, 0] = cx[ind]
    xref[1, 0] = cy[ind]
    xref[2, 0] = sp[ind]
    xref[3, 0] = cyaw[ind]
    dref[0, 0] = 0.0  # steer operational point should be 0

    S = 0.0
    vel=state.v
    for i in range(T + 1):
        S += abs(vel) * DT
        vel+=acc*DT
        dind = int(round(travel / dl))

        if (ind + dind) < ncourse:
            xref[0, i] = cx[ind + dind]
            xref[1, i] = cy[ind + dind]
            xref[2, i] = sp[ind + dind]
            xref[3, i] = cyaw[ind + dind]
            dref[0, i] = 0.0
        else:
            xref[0, i] = cx[ncourse - 1]
            xref[1, i] = cy[ncourse - 1]
            xref[2, i] = sp[ncourse - 1]
            xref[3, i] = cyaw[ncourse - 1]
            dref[0, i] = 0.0

    return xref, ind, dref

def calc_ref_trajectory(state, ccx, ccy, ccyaw, ck, ssp, dl, ind):
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    #ind, _ = get_closest_point(state, ccx, ccy, ccyaw, pind)
    #if pind >= ind:
    #    ind = pind

    xref[0, 0] = ccx[0]
    xref[1, 0] = ccy[0]
    xref[2, 0] = ssp[0]
    xref[3, 0] = ccyaw[0]
    dref[0, 0] = 0.0  # steer operational point should be 0

    travel = 0.0

    for i in range(T + 1):
        travel += abs(state.v) * DT
        dind = int(round(travel / dl))

        if (dind) < len(ccx)-1:
            xref[0, i] = ccx[dind]
            xref[1, i] = ccy[dind]
            xref[2, i] = ssp[dind]
            xref[3, i] = ccyaw[dind]
            dref[0, i] = 0.0
        else:
            xref[0, i] = ccx[-1]
            xref[1, i] = ccy[-1]
            xref[2, i] = ssp[- 1]
            xref[3, i] = ccyaw[- 1]
            dref[0, i] = 0.0

    return xref, ind, dref

def check_goal(state, goal, tind, nind):

    # check goal
    dx = state.x - goal[0]
    dy = state.y - goal[1]
    d = math.sqrt(dx ** 2 + dy ** 2)

    isgoal = (d <= GOAL_DIS)

    if abs(tind - nind) >= 5:
        isgoal = False

    isstop = (abs(state.v) <= STOP_SPEED)

    if isgoal and isstop:
        return True

    return False




def calc_speed_profile(cx, cy, cyaw, target_speed):

    speed_profile = [target_speed] * len(cx)
    direction = 1.0  # forward

    # Set stop point
    for i in range(len(cx) - 1):
        dx = cx[i + 1] - cx[i]
        dy = cy[i + 1] - cy[i]

        move_direction = math.atan2(dy, dx)

        if dx != 0.0 and dy != 0.0:
            dangle = abs(pi_2_pi(move_direction - cyaw[i]))
            if dangle >= math.pi / 4.0:
                direction = -1.0
            else:
                direction = 1.0

        if direction != 1.0:
            speed_profile[i] = - target_speed
        else:
            speed_profile[i] = target_speed

    speed_profile[-1] = 0.0

    return speed_profile


def smooth_yaw(yaw):

    for i in range(len(yaw) - 1):
        dyaw = yaw[i + 1] - yaw[i]

        while dyaw >= math.pi / 2.0:
            yaw[i + 1] -= math.pi * 2.0
            dyaw = yaw[i + 1] - yaw[i]

        while dyaw <= -math.pi / 2.0:
            yaw[i + 1] += math.pi * 2.0
            dyaw = yaw[i + 1] - yaw[i]

    return yaw


def get_straight_course(dl):
    ax = [0.0, 5.0, 10.0, 20.0, 30.0, 40.0, 50.0]
    ay = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    return cx, cy, cyaw, ck


def get_straight_course2(dl):
    ax = [0.0, -10.0, -20.0, -40.0, -50.0, -60.0, -70.0]
    ay = [0.0, -1.0, 1.0, 0.0, -1.0, 1.0, 0.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    return cx, cy, cyaw, ck


def get_straight_course3(dl):
    ax = [0.0, -10.0, -20.0, -40.0, -50.0, -60.0, -70.0]
    ay = [0.0, -1.0, 1.0, 0.0, -1.0, 1.0, 0.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    cyaw = [i - math.pi for i in cyaw]

    return cx, cy, cyaw, ck


def get_forward_course(dl):
    ax = [0.0, 60.0, 125.0, 50.0, 75.0, 30.0, -10.0]
    ay = [0.0, 0.0, 50.0, 65.0, 30.0, 50.0, -20.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    return cx, cy, cyaw, ck


def get_switch_back_course(dl):
    ax = [0.0, 30.0, 6.0, 20.0, 35.0]
    ay = [0.0, 0.0, 20.0, 35.0, 20.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)
    ax = [35.0, 10.0, 0.0, 0.0]
    ay = [20.0, 30.0, 5.0, 0.0]
    cx2, cy2, cyaw2, ck2, s2 = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)
    cyaw2 = [i - math.pi for i in cyaw2]
    cx.extend(cx2)
    cy.extend(cy2)
    cyaw.extend(cyaw2)
    ck.extend(ck2)

    return cx, cy, cyaw, ck


def main():
    

    t, x, y, yaw, v, d, a = do_simulation(
        cx, cy, cyaw, ck, sp, dl, initial_state)




def main2():
    print(__file__ + " start!!")

    dl = 1.0  # course tick
    cx, cy, cyaw, ck = get_straight_course3(dl)

    sp = calc_speed_profile(cx, cy, cyaw, TARGET_SPEED)

    initial_state = State(x=cx[0], y=cy[0], yaw=0.0, v=0.0)

    t, x, y, yaw, v, d, a = do_simulation(
        cx, cy, cyaw, ck, sp, dl, initial_state)

    if show_animation:  # pragma: no cover
        plt.close("all")
        plt.subplots()
        plt.plot(cx, cy, "-r", label="spline")
        plt.plot(x, y, "-g", label="tracking")
        plt.grid(True)
        plt.axis("equal")
        plt.xlabel("x[m]")
        plt.ylabel("y[m]")
        plt.legend()

        plt.subplots()
        plt.plot(t, v, "-r", label="speed")
        plt.grid(True)
        plt.xlabel("Time [s]")
        plt.ylabel("Speed [kmh]")

        plt.show()

In [2]:
def iterative_linear_mpc_control(xref, Motion_Status, dref, oa, od,x0):
    """
    MPC contorl with updating operational point iteraitvely
    """  
    
    if oa is None or od is None:
        oa = [0.0] * T
        od = [0.0] * T
    state=Motion_Status.get_state()   
    for i in range(MAX_ITER):
        xbar = Motion_Status.get_trajectory(oa, od,T)
        poa, pod = oa[:], od[:]
        oa, od, ox, oy, oyaw, ov = linear_mpc_control(xref, xbar, state, dref,x0)
        du = sum(abs(oa - poa)) + sum(abs(od - pod))  # calc u change value
        if du <= DU_TH:
            break
    else:
        print("Iterative is max iter")
    
    return oa, od, ox, oy, oyaw, ov,xbar

def get_linear_model_matrix(v, phi, delta):

    A = np.zeros((NX, NX))
    A[0, 0] = 1.0
    A[1, 1] = 1.0
    A[2, 2] = 1.0
    A[3, 3] = 1.0
    A[0, 2] = DT * math.cos(phi)
    A[0, 3] = - DT * v * math.sin(phi)#
    A[1, 2] = DT * math.sin(phi)
    A[1, 3] = DT * v * math.cos(phi)#
    A[3, 2] = DT * math.tan(delta) / WB

    B = np.zeros((NX, NU))
    B[2, 0] = DT
    B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)#

    C = np.zeros(NX)
    C[0] = DT * v * math.sin(phi) * phi
    C[1] = - DT * v * math.cos(phi) * phi
    C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)
    #C = np.zeros(NX)
    return A, B, C

def linear_mpc_control(xref, xbar, state, dref,x0):
    """
    linear mpc control

    xref: reference point
    xbar: operational point
    x0: initial state
    dref: reference steer angle
    """

    x = cvxpy.Variable((NX, T + 1))
    u = cvxpy.Variable((NU, T))

    cost = 0.0
    constraints = []

    for t in range(T):
        cost += cvxpy.quad_form(u[:, t], R)

        if t != 0:
            cost += cvxpy.quad_form(xref[:, t] - x[:, t], Q)

        A, B, C = get_linear_model_matrix(xbar[2, t], xbar[3, t], 0)
        constraints += [x[:, t + 1] == A * x[:, t] + B * u[:, t]+ C]

        if t < (T - 1):
            cost += cvxpy.quad_form(u[:, t + 1] - u[:, t], Rd)
            constraints += [cvxpy.abs(u[1, t + 1] - u[1, t]) <=
                            MAX_DSTEER * DT]

    cost += cvxpy.quad_form(xref[:, T] - x[:, T], Qf)

    constraints += [x[:, 0] == [state.x,state.y,state.v,state.yaw]]
    constraints += [x[2, :] <= MAX_SPEED]
    constraints += [x[2, :] >= MIN_SPEED]
    constraints += [cvxpy.abs(u[0, :]) <= MAX_ACCEL]
    constraints += [cvxpy.abs(u[1, :]) <= MAX_STEER]

    prob = cvxpy.Problem(cvxpy.Minimize(cost), constraints)
    prob.solve(solver=cvxpy.ECOS, verbose=False)

    if prob.status == cvxpy.OPTIMAL or prob.status == cvxpy.OPTIMAL_INACCURATE:
        ox = get_nparray_from_matrix(x.value[0, :])
        oy = get_nparray_from_matrix(x.value[1, :])
        ov = get_nparray_from_matrix(x.value[2, :])
        oyaw = get_nparray_from_matrix(x.value[3, :])
        oa = get_nparray_from_matrix(u.value[0, :])
        odelta = get_nparray_from_matrix(u.value[1, :])

    else:
        print("Error: Cannot solve mpc..")
        oa, odelta, ox, oy, oyaw, ov = None, None, None, None, None, None

    return oa, odelta, ox, oy, oyaw, ov



In [3]:
class State:
     
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.predelta = None

class Motion():

    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.state=State(x=x, y=y, yaw=yaw, v=v)
        
    def get_state(self):
        return State(x=self.state.x, y=self.state.y, yaw=self.state.yaw, v=self.state.v)
        
    def set_state(self,state):
        self.state=state
    def get_next_state(self, a, delta):

        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER

        next_x = self.state.x + self.state.v * math.cos(self.state.yaw) * DT
        next_y = self.state.y + self.state.v * math.sin(self.state.yaw) * DT
        next_yaw = self.state.yaw + self.state.v / WB * math.tan(delta) * DT
        next_v = self.state.v + a * DT
        
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw)


    def get_trajectory(self,accels, deltas,time_step):
        xbar =np.zeros((NX, time_step + 1))
        xbar[:, 0] = self.state.x,self.state.y,self.state.v,self.state.yaw
        #_state=State(x=state.x,y=state.y,v=state.v,yaw=state.yaw)
        _state=Motion.get_state(self)
        for (accel, delta, i) in zip(accels, deltas, range(1, T + 1)):
            next_state = Motion.get_next_state(self,accel, delta)
            xbar[:, i] = next_state.x,next_state.y,next_state.v,next_state.yaw

        return xbar


In [4]:
def gen_points(cx,cy):
    dx=20#2
    dy=10#1
    pts=[[cx-dx,cy+dy],
    [cx+dx,cy+dy],
    [cx+dx,cy-dy],
    [cx-dx,cy-dy],
    [cx-dx,cy+dy]]
    pts=np.array(pts,dtype=np.float64)
    return pts
class move_points():
    def __init__(self,points,XX,YY,phi):
        self.points=points.copy()
        self.XX=XX
        self.YY=YY
        self.ox=0
        self.oy=0
        phi=(math.pi*phi)/180
        self.rot_mat=np.array([[math.cos(phi),-math.sin(phi)],[math.sin(phi),math.cos(phi)]])
    def move(self):
        self.points[:,0]+=self.XX
        self.points[:,1]+=self.YY
    def get_origin(self):
        ox=0
        oy=0
        oo=np.mean(self.points,axis=0)
        self.ox=oo[0]
        self.oy=oo[1]
        print(self.ox,self.oy)
        return self.ox,self.oy
    def shift_origin(self):
        self.points[:,0]-=np.float64(self.ox)
        self.points[:,1]-=np.float64(self.oy)
        return self.points
    def reverse_shift_origin(self):
        self.points[:,0]+=self.ox
        self.points[:,1]+=self.oy
        return self.points
    def rotate(self):
        for i in range(len(self.points)):
            self.points[i]=self.points[i].dot(self.rot_mat)
        return self.points
    def apply(self):
        self.move()
        self.get_origin()
        self.shift_origin()
        self.rotate()
        self.reverse_shift_origin()
        return self.points

In [ ]:
%matplotlib tk
import gym
from matplotlib import pyplot as plt
import cv2
import numpy as np
from car_env import CarRacing
import time
import cubic_spline_planner
def rad_to_deg():
    pass
def accl_to_gas(accl,vel):
    SIZE = 0.02
    ENGINE_POWER= 100000000*SIZE*SIZE
    WHEEL_MOMENT_OF_INERTIA = 4000*SIZE*SIZE
    WHEEL_R  = 27 
    wheel_rad=WHEEL_R*SIZE
    mass=(2*WHEEL_MOMENT_OF_INERTIA)/wheel_rad
    #mass=.25
    power=4*mass*accl*(vel+accl/2)
    gas=power/ENGINE_POWER
    return gas

def env_step(env,action,show_plot=False):
   
    frame=np.zeros((100,200,3),dtype=np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    nst,rw,ter,info=envc.step(action) # take a random action
    frame[0:96,0:96,:]=np.array(nst)
    
    pos=envc.car.hull.position
    vel=np.sqrt(envc.car.hull.linearVelocity[0]**2+envc.car.hull.linearVelocity[1]**2)
    ang=envc.car.hull.angle
    if show_plot==True:
        plt.cla()
        img = cv2.putText(np.array(frame), 'vel %f'%vel, (100,10), font,.3, (255,255,0),1, cv2.LINE_AA)
        img = cv2.putText(np.array(img), 'ang %f'%ang, (100,20), font,.3, (255,0,255),1, cv2.LINE_AA)
        image = cv2.putText(np.array(img), 'x,y %f,%f'%(pos[0],pos[1]), (100,30), font,.2, (255,0,255),1, cv2.LINE_AA)
       
        plt.imshow(image)
        plt.show()
        plt.pause(0.0001)
        
    return [pos[0],pos[1],ang,vel],rw,ter,info
#envc.close()


envc=CarRacing()
def env_reset(envc):
    import pickle
    envc.reset()
    save=False
    if save==True:
        f=open('path-1','wb')
        pickle.dump(envc.track,f)
        f.close()
    f=open('path-1','rb')
    track=pickle.load(f)
    f.close()
    envc.track=track
    
    points=[[x,y] for a,b,x,y in envc.track]
    points=np.array(points)
    plt.plot(points[:,0],points[:,1])
    plt.show()
    return [env_step(envc,[0,0,0]),points]
    
#st,points=env_reset(envc)
'''
for i in range(100):
    st,rw,ter,info=env_step(envc,[0,1,0])
    time.sleep(0.1)
    #print(st)
    '''
#envc.close()



'\nfor i in range(100):\n    st,rw,ter,info=env_step(envc,[0,1,0])\n    time.sleep(0.1)\n    #print(st)\n    '

In [ ]:
envc.close()

In [ ]:
#cx, cy, cyaw, ck = get_switch_back_course(dl)
envc=CarRacing()
[init_state,traj_points]=env_reset(envc)
ix,iy,iyaw,ivel=init_state[0]
tx,ty=traj_points[:50,0],traj_points[:50,1]

cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=1.0)
yaw0=cyaw[0]
cyaw=[yaw-yaw0 for yaw in cyaw]
#dcx=np.gradient(cx)
#dcy=np.gradient(cy)
#cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]
sp = calc_speed_profile(cx, cy, cyaw, TARGET_SPEED)


Motion_Status = Motion(x=ix, y=iy, yaw=iyaw, v=ivel)
state=Motion_Status.get_state()
print(state)

Track generation: 1345..1686 -> 341-tiles track


In [ ]:
def get_closest_point(state, ccx, ccy, cyaw, pind):
    #print(ccx,ccy,state.x,state.y)
    d = [(state.x-idx) ** 2 + (state.y-idy) ** 2 for (idx, idy) in zip(ccx,ccy)]
    #print('printing d',d,len(d),type(d))
    mind = min(d)
    ind = d.index(mind) + pind
    #print('ds',ind,mind)
    return ind, mind

def calc_ref_trajectory(state, ccx, ccy, ccyaw, ck, ssp, dl, ind):
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    #ind, _ = get_closest_point(state, ccx, ccy, ccyaw, pind)
    #if pind >= ind:
    #    ind = pind

    xref[0, 0] = ccx[0]
    xref[1, 0] = ccy[0]
    xref[2, 0] = ssp[0]
    xref[3, 0] = ccyaw[0]
    dref[0, 0] = 0.0  # steer operational point should be 0

    travel = 0.0

    for i in range(T + 1):
        travel += abs(state.v) * DT
        dind = int(round(travel / dl))

        if (dind) < len(ccx)-1:
            xref[0, i] = ccx[dind]
            xref[1, i] = ccy[dind]
            xref[2, i] = ssp[dind]
            xref[3, i] = ccyaw[dind]
            dref[0, i] = 0.0
        else:
            xref[0, i] = ccx[-1]
            xref[1, i] = ccy[-1]
            xref[2, i] = ssp[- 1]
            xref[3, i] = ccyaw[- 1]
            dref[0, i] = 0.0

    return xref, ind, dref

In [ ]:
# initial yaw compensation
if state.yaw - cyaw[0] >= math.pi:
        state.yaw -= math.pi * 2.0
elif state.yaw - cyaw[0] <= -math.pi:
        state.yaw += math.pi * 2.0
Motion_Status.set_state(state)
goal = [cx[-1], cy[-1]]

In [ ]:
dl=1.0
time = 0.0
x = [state.x]
y = [state.y]
yaw = [state.yaw]
v = [state.v]
t = [0.0]
d = [0.0]
a = [0.0]
def get_raw_sensor_value():
    pass
_cx=cx[:10]
_cy=cy[:10]
_cyaw=cyaw[:10]
_sp=sp[:10]
#index, _ = get_closest_point(state, _cx, _cy, cyaw, 0)
odelta, oa = None, None
index=0
cyaw = smooth_yaw(cyaw)
accl,gas,steer=0,0,0
while MAX_TIME >= time:
        state=Motion_Status.get_state()
        print("x,y,yaw,v\n",state.x,state.y,state.yaw,state.v)
        
        index, _ = get_closest_point(state, _cx, _cy, cyaw, index)
        #if pind >= index:
        #    index = pind
        _cx=cx[index:index+15]
        _cy=cy[index:index+15]
        _cyaw=cyaw[index:index+15]
        _sp=sp[index:index+15]
        
        pind=index
        xref,_, dref = calc_ref_trajectory(state, _cx, _cy,_cyaw, ck, _sp, dl, index)
        print('xref...............\n',xref,'\n')
        x0 = [state.x, state.y, state.v, state.yaw]  # current state

        oa, odelta, ox, oy, oyaw, ov ,xbar= iterative_linear_mpc_control(
            xref, Motion_Status, dref, oa, odelta,x0)

        if odelta is not None:
            di, ai = odelta[0], oa[0]
        accl=ai
        gas=accl_to_gas(accl,state.v)
        steer=di
        print('accl,gas,steer\n',accl,gas,steer)
        nst,rw,ter,info=env_step(envc,[steer,gas,0])
        next_state=State(x=nst[0],y=nst[1],yaw=nst[2],v=nst[3])
        #next_state = Motion_Status.get_next_state(ai, di)
        Motion_Status.set_state(next_state)
        
        time = time + DT

        x.append(state.x)
        y.append(state.y)
        yaw.append(state.yaw)
        v.append(state.v)
        t.append(time)
        d.append(di)
        a.append(ai)

        if check_goal(state, goal, index, len(cx)):
            print("Goal")
            break

        if show_animation:  # pragma: no cover
            plt.cla()
            if ox is not None:
                plt.plot(ox, oy, "xr", label="MPC")
            plt.plot(cx, cy, "-r", label="course")
            plt.plot(x, y, "ob", label="trajectory")
            plt.plot(xref[0, :], xref[1, :], "xk", label="xref")
            plt.plot(cx[index], cy[index], "xg", label="target")
            #plot_car(state.x, state.y, state.yaw, steer=di)
            
            points=gen_points(state.x,state.y)
            ang_yaw=-(state.yaw*180.0)/math.pi
            rp=move_points(points,0,0,ang_yaw)
            r_points=rp.apply()
            points=np.array(points,dtype=np.float64)
            r_points=np.array(r_points,dtype=np.float64)
            plt.plot(r_points[:,0].flatten(),r_points[:,1].flatten(),"-r")
            
            plt.axis("equal")
            plt.grid(True)
            plt.title("Time[s]:" + str(round(time, 2))
                      + ", speed[km/h]:" + str(round(state.v * 3.6, 2))
                      + '\nyaw'+str(state.yaw)
                      + '\nxref_yaw'+str(xref[3][0])
                      + '\nsteer'+str(steer))
            plt.pause(0.0001)

if show_animation:  # pragma: no cover
            plt.close("all")
            plt.subplots()
            plt.plot(cx, cy, "-r", label="spline")
            plt.plot(x, y, "-g", label="tracking")
            plt.grid(True)
            plt.axis("equal")
            plt.xlabel("x[m]")
            plt.ylabel("y[m]")
            plt.legend()

            plt.subplots()
            plt.plot(t, v, "-r", label="speed")
            plt.grid(True)
            plt.xlabel("Time [s]")
            plt.ylabel("Speed [kmh]")

            plt.show()


x,y,yaw,v
 220.1962432861328 -52.31069564819336 -0.0879134088754654 7.205593910058988e-07
xref...............
 [[ 2.19738705e+02  2.19738705e+02  2.19738705e+02  2.19738705e+02
   2.19738705e+02  2.19738705e+02]
 [-5.20321233e+01 -5.20321233e+01 -5.20321233e+01 -5.20321233e+01
  -5.20321233e+01 -5.20321233e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.02288844e-10  1.02288844e-10  1.02288844e-10  1.02288844e-10
   1.02288844e-10  1.02288844e-10]] 

accl,gas,steer
 -1.0000000003112208 0.00029629586948256567 -0.002486451932849256
216.1962432861328 -50.31069564819336
x,y,yaw,v
 220.19625854492188 -52.310550689697266 -0.08790624141693115 0.0071871631277741314
xref...............
 [[ 2.19738705e+02  2.19738705e+02  2.19738705e+02  2.19738705e+02
   2.19738705e+02  2.19738705e+02]
 [-5.20321233e+01 -5.20321233e+01 -5.20321233e+01 -5.20321233e+01
  -5.20321233e+01 -5.20321233e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+0

x,y,yaw,v
 220.19715881347656 -52.30022048950195 -0.08790390193462372 0.07221741223317156
xref...............
 [[ 2.19738705e+02  2.19738705e+02  2.19738705e+02  2.19738705e+02
   2.19738705e+02  2.19738705e+02]
 [-5.20321233e+01 -5.20321233e+01 -5.20321233e+01 -5.20321233e+01
  -5.20321233e+01 -5.20321233e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.02288844e-10  1.02288844e-10  1.02288844e-10  1.02288844e-10
   1.02288844e-10  1.02288844e-10]] 

accl,gas,steer
 -1.0000000001084661 0.00025350079281034746 -0.0013112320973675516
216.19715881347656 -50.30022048950195
x,y,yaw,v
 220.19729614257812 -52.298683166503906 -0.08790353685617447 0.07718148734467171
xref...............
 [[ 2.19738705e+02  2.19738705e+02  2.19738705e+02  2.19738705e+02
   2.19738705e+02  2.19738705e+02]
 [-5.20321233e+01 -5.20321233e+01 -5.20321233e+01 -5.20321233e+01
  -5.20321233e+01 -5.20321233e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+0

x,y,yaw,v
 220.19956970214844 -52.27296829223633 -0.08789901435375214 0.1319016201309178
xref...............
 [[ 2.19738705e+02  2.19738705e+02  2.19738705e+02  2.19738705e+02
   2.19738705e+02  2.19738705e+02]
 [-5.20321233e+01 -5.20321233e+01 -5.20321233e+01 -5.20321233e+01
  -5.20321233e+01 -5.20321233e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.02288844e-10  1.02288844e-10  1.02288844e-10  1.02288844e-10
   1.02288844e-10  1.02288844e-10]] 

accl,gas,steer
 -1.0000000000937934 0.00021813237330400246 -3.727099091575214e-05
216.19956970214844 -50.27296829223633
x,y,yaw,v
 220.19981384277344 -52.27025604248047 -0.08789891749620438 0.1360872263474596
xref...............
 [[ 2.19738705e+02  2.19738705e+02  2.19738705e+02  2.19738705e+02
   2.19738705e+02  2.19738705e+02]
 [-5.20321233e+01 -5.20321233e+01 -5.20321233e+01 -5.20321233e+01
  -5.20321233e+01 -5.20321233e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -

x,y,yaw,v
 220.20323181152344 -52.231590270996094 -0.08790340274572372 0.18237387363748195
xref...............
 [[ 2.19738705e+02  2.19738705e+02  2.19738705e+02  2.19738705e+02
   2.19738705e+02  2.19738705e+02]
 [-5.20321233e+01 -5.20321233e+01 -5.20321233e+01 -5.20321233e+01
  -5.20321233e+01 -5.20321233e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.02288844e-10  1.02288844e-10  1.02288844e-10  1.02288844e-10
   1.02288844e-10  1.02288844e-10]] 

accl,gas,steer
 -1.000000000233203 0.00018822288980929833 0.001227074659373873
216.20323181152344 -50.231590270996094
x,y,yaw,v
 220.2035675048828 -52.22788619995117 -0.08790458738803864 0.18592522341667087
xref...............
 [[ 2.19738705e+02  2.19738705e+02  2.19738705e+02  2.19738705e+02
   2.19738705e+02  2.19738705e+02]
 [-5.20321233e+01 -5.20321233e+01 -5.20321233e+01 -5.20321233e+01
  -5.20321233e+01 -5.20321233e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -

x,y,yaw,v
 220.20797729492188 -52.178226470947266 -0.08792679011821747 0.22529795147313653
xref...............
 [[ 2.19738705e+02  2.19738705e+02  2.19738705e+02  2.19738705e+02
   2.19738705e+02  2.19738705e+02]
 [-5.20321233e+01 -5.20321233e+01 -5.20321233e+01 -5.20321233e+01
  -5.20321233e+01 -5.20321233e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.02288844e-10  1.02288844e-10  1.02288844e-10  1.02288844e-10
   1.02288844e-10  1.02288844e-10]] 

accl,gas,steer
 -1.0000000000843867 0.00016278639916577066 0.002359506375169141
216.20797729492188 -50.178226470947266
x,y,yaw,v
 220.20838928222656 -52.17367935180664 -0.0879296362400055 0.2283262777029587
xref...............
 [[ 2.19738705e+02  2.19738705e+02  2.19738705e+02  2.19738705e+02
   2.19738705e+02  2.19738705e+02]
 [-5.20321233e+01 -5.20321233e+01 -5.20321233e+01 -5.20321233e+01
  -5.20321233e+01 -5.20321233e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -

...

accl,gas,steer
 -1.0000000011579333 3.7062989314006666e-08 0.03302801292988031
218.44732666015625 -34.612483978271484
x,y,yaw,v
 222.44937133789062 -36.60269546508789 -0.18915797770023346 0.49993823620622285
xref...............
 [[ 2.21360989e+02  2.21360989e+02  2.21360989e+02  2.21360989e+02
   2.21360989e+02  2.21462382e+02]
 [-3.61145797e+01 -3.61145797e+01 -3.61145797e+01 -3.61145797e+01
  -3.61145797e+01 -3.51197332e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.64969405e-08  1.64969405e-08  1.64969405e-08  1.64969405e-08
   1.64969405e-08 -1.18371088e-08]] 

accl,gas,steer
 -1.0000000011684114 3.660111292148006e-08 0.03285510241011446
218.44937133789062 -34.60269546508789
x,y,yaw,v
 222.451416015625 -36.5929069519043 -0.18925924599170685 0.49993915181972043
xref...............
 [[ 2.21360989e+02  2.21360989e+02  2.21360989e+02  2.21360989e+02
   2.21360989e+02  2.21462382e+02]
 [-3.61145797e+01 -3.61145797e+01 -3.61

accl,gas,steer
 -1.0000000015747472 2.7694559581289356e-08 0.030778979041156437
218.473876953125 -34.485252380371094
x,y,yaw,v
 222.47593688964844 -36.4754638671875 -0.19044512510299683 0.4999546478411152
xref...............
 [[ 2.21360989e+02  2.21360989e+02  2.21360989e+02  2.21360989e+02
   2.21360989e+02  2.21462382e+02]
 [-3.61145797e+01 -3.61145797e+01 -3.61145797e+01 -3.61145797e+01
  -3.61145797e+01 -3.51197332e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.64969405e-08  1.64969405e-08  1.64969405e-08  1.64969405e-08
   1.64969405e-08 -1.18371088e-08]] 

accl,gas,steer
 -1.0000000016354982 2.687583804921961e-08 0.030605628598732615
218.47593688964844 -34.4754638671875
x,y,yaw,v
 222.4779815673828 -36.46567916870117 -0.1905408799648285 0.4999560025478877
xref...............
 [[ 2.21360989e+02  2.21360989e+02  2.21360989e+02  2.21360989e+02
   2.21360989e+02  2.21462382e+02]
 [-3.61145797e+01 -3.61145797e+01 -3.61145

accl,gas,steer
 -1.000000000102123 1.7842428658967303e-08 0.028523178932566083
218.5005645751953 -34.3580207824707
x,y,yaw,v
 222.5026092529297 -36.34823226928711 -0.19164936244487762 0.49997106268487435
xref...............
 [[ 2.21360989e+02  2.21360989e+02  2.21360989e+02  2.21360989e+02
   2.21360989e+02  2.21462382e+02]
 [-3.61145797e+01 -3.61145797e+01 -3.61145797e+01 -3.61145797e+01
  -3.61145797e+01 -3.51197332e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.64969405e-08  1.64969405e-08  1.64969405e-08  1.64969405e-08
   1.64969405e-08 -1.18371088e-08]] 

accl,gas,steer
 -1.0000000001151537 1.714807271455934e-08 0.02834927462657404
218.5026092529297 -34.34823226928711
x,y,yaw,v
 222.50466918945312 -36.338443756103516 -0.19173824787139893 0.49997225815737284
xref...............
 [[ 2.21360989e+02  2.21360989e+02  2.21360989e+02  2.21360989e+02
   2.21360989e+02  2.21462382e+02]
 [-3.61145797e+01 -3.61145797e+01 -3.611

accl,gas,steer
 -1.0000000006211998 9.5437045258285e-09 0.026258530818278877
218.52723693847656 -34.23078536987305
x,y,yaw,v
 222.52928161621094 -36.22100067138672 -0.19276349246501923 0.49998485386385516
xref...............
 [[ 2.21360989e+02  2.21360989e+02  2.21360989e+02  2.21360989e+02
   2.21360989e+02  2.21462382e+02]
 [-3.61145797e+01 -3.61145797e+01 -3.61145797e+01 -3.61145797e+01
  -3.61145797e+01 -3.51197332e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.64969405e-08  1.64969405e-08  1.64969405e-08  1.64969405e-08
   1.64969405e-08 -1.18371088e-08]] 

accl,gas,steer
 -1.0000000007429246 8.975708218309415e-09 0.02608404627811106
218.52928161621094 -34.22100067138672
x,y,yaw,v
 222.53134155273438 -36.211212158203125 -0.192845419049263 0.4999858683440325
xref...............
 [[ 2.21360989e+02  2.21360989e+02  2.21360989e+02  2.21360989e+02
   2.21360989e+02  2.21462382e+02]
 [-3.61145797e+01 -3.61145797e+01 -3.6114

accl,gas,steer
 -1.0000000002386393 2.721344462326476e-09 0.023986242834297067
218.55389404296875 -34.103553771972656
x,y,yaw,v
 222.5559539794922 -36.09376525878906 -0.19378580152988434 0.4999961533436648
xref...............
 [[ 2.21360989e+02  2.21360989e+02  2.21360989e+02  2.21360989e+02
   2.21360989e+02  2.21462382e+02]
 [-3.61145797e+01 -3.61145797e+01 -3.61145797e+01 -3.61145797e+01
  -3.61145797e+01 -3.51197332e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.64969405e-08  1.64969405e-08  1.64969405e-08  1.64969405e-08
   1.64969405e-08 -1.18371088e-08]] 

accl,gas,steer
 -1.0000000004050098 2.2796200543064843e-09 0.023811000086024603
218.5559539794922 -34.09376525878906
x,y,yaw,v
 222.55799865722656 -36.08397674560547 -0.1938607394695282 0.49999695633542707
xref...............
 [[ 2.21360989e+02  2.21360989e+02  2.21360989e+02  2.21360989e+02
   2.21360989e+02  2.21462382e+02]
 [-3.61145797e+01 -3.61145797e+01 -3.6

In [ ]:
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=1.0)
i_yaw=cyaw[0]
cyaw=[yaw-i_yaw for yaw in cyaw]
cyaw

In [ ]:
#%matplotlib inline
plt.plot(cx,cy)
plt.show()

In [ ]:
plt.plot(range(len(cyaw)),cyaw)
plt.show()

In [ ]:
dcx=np.gradient(cx)
dcy=np.gradient(cy)
cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]

In [ ]:
cyaw